# Measurement: Linked SED and UMETRICS Data

Brian Kim, Maryah Garner, Rukhshan Mian, Benjamin Feder, Allison Nunez, Ekaterina Levitskaya.



## Introduction 

The information available in datasets don't just arise naturally. They are collected through various different means, and these processes can have implications on the types of conclusions you can make. For example, the SED and SDR are surveys, and the SDR involves a complex sampling design and requires the use of survey weights. The UMETRICS data are built using administrative records of grants provided by IRIS-member universities.

What are the possible implications of this? For one, the SED directly measures funding by asking whether the respondent received grant funding during their time in graduate school. However, this is a very high-level measure, and the more fine-grained information about sources of funding and consistency of funding are missing. For example, there is nothing on the SED that asks about federal funding. In addition, there are a few possible sources of error. The respondent may not remember exactly when and how much funding they had or they may have misunderstood the question. The UMETRICS data avoid these issues of measurement, but there are possible sources of error and missingness there as well. Not all universities participate in IRIS, and so only a subset of the universities are present in UMETRICS data. Bringing in the UMETRICS data to use with the SED means that the datasets must be linked, which comes with linkage errors.

While the SED and SDR questions were designed for research purposes, the UMETRICS data was not generated with research in mind. When using administrative data like UMETRICS for research, you have to thoughtfully construct measures from the data that is provided. In this notebook, you will explore the linked SED-UMETRICS data to get a better sense of how these two data sources might be used together and to think critically about what exactly is being measured and how missingness in the data should be interpreted. 

### Learning Objectives

* Use the linked SED-UMETRICS data.

* Critically evaluate the presence of missingness in data.


## R Setup

As always, start by importing the required libraries, as well as creating the connection to the database.

In [ ]:
# Switching off warnings
options(warn = -1)

# Database interaction imports
suppressMessages(library(odbc))

# for data manipulation/visualization
suppressMessages(library(tidyverse))

# scaling data, calculating percentages, overriding default graphing
suppressMessages(library(scales))

# add weights to data
suppressMessages(library(survey))

#Switching on warnings
options(warn = 0)

# to better view images
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
theme_set(theme_gray(base_size = 24))
options(repr.plot.width = 20, repr.plot.height = 12)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## Using the SED-UMETRICS Linked Data

In the `02_Data_Exploration_UMETRICS` notebook we used an inner join to link SED and UMETRICS data.

An inner join provides only the people who are in both data sets. That means if someone graduated and took the SED, but didn't receive any funding, that person would not show up. This is important if you want to, for example, look at how federal funding affects career pathways -- without a comparison group of people who did not receive funding, it would be very hard to make conclusions about the people who did get funding.

An individual might not show up in the UMETRICS dataset for the following reasons:
- They did not attend a UMETRICS university - their institution data is by default not included in the UMETRICS dataset.
- They attended a UMETRICS university, but they are not found in the UMETRICS dataset - in this case, because their institution does provide data for UMETRICS, we could assume that they did not show up in the dataset because they did not receive funding. However, not everyone missing might be someone without funding. One other possible explanation could be that they were not found due to the record linkage processes.

To construct a measure which compares those individuals who received vs did not receive funding, we would need to subset the SED to all those individuals who graduated from UMETRICS universities. We do this by using an `INNER JOIN` to link institutions from the SED to institutions listed in the **tr_ncses_2021.dbo.umetrics_inst_xwalk**, which contains institute codes for UMETRICS universities. This leaves us with all SED observations associated with a UMETRICS university. We use a `LEFT JOIN` to bring in information on individual emp_number and subsequently use another `LEFT JOIN` to bring in information from **ds_iris_umetrics.dbo.semester** (using our new emp_number information).

In [ ]:
# Left join SED with UMETRICS semester table and filter to only UMETRICS universities in the SED
qry <- " 
select sed.drf_id, semester, srceprim, sed.phdinst
from ds_nsf_ncses.dbo.nsf_sed sed 
INNER JOIN tr_ncses_2021.dbo.umetrics_inst_xwalk inst_xwalk
on sed.phdinst = inst_xwalk.phdinst
LEFT JOIN tr_ncses_2021.dbo.sed_umetrics_xwalk indiv_xwalk
on indiv_xwalk.drf_id = sed.drf_id
LEFT JOIN ds_iris_umetrics.dbo.semester sem
on sem.emp_number = indiv_xwalk.emp_number
where sed.phdfy = '2015'"

umetrics_sed_support <- dbGetQuery(con, qry)

Now that we have a table with everyone who attended UMETRICS universities, we can create a flag for whether they received funding or not (i.e. show up in the UMETRICS dataset or not)

Since we are using a `LEFT JOIN` to bring in the **semester** data table, the new data frame will have at least one observaton from everyone in **SED** from the **UMETRICS Instutituons**. The individuals who are not found in the UMETRICS semester table, will have missing values in the **semester** variable. Based on this, we can create a flag called **in_umetrics** for those individuals with and without funding (i.e. who show up in UMETRICS or who do not).
> Note: for individuals who are in the **semester** data table, there will be an observation for each semester they were on a grant, and their information from SED will be repeated with each observation. 

In [ ]:
# Create a new column called "umetrics_category" based on whether the "semester" variable is missing,
# and whether IPEDS ID is found in the UMETRICS institution crosswalk
umetrics_sed_support <-umetrics_sed_support %>%
    mutate(in_umetrics =
          case_when(!is.na(semester) ~ 'on a grant',
                    is.na(semester) ~ 'not on a grant'))

Now that we know who received funding and who did not, we can explore the proportions by the primary source of support.

First, we need to recode the **srceprim** variable to the actual description of the source of support.

In [ ]:
# unique observations for srceprim
unique(umetrics_sed_support$srceprim)

Notice that there are **srceprim** values coded as an empty string `''`, which means the source of support is "Unknown"

In [ ]:
# Create a new variable "source_cat" with the description of the support from the SED data dictionary
# (make sure to include empty string values in the categorization as an "Unknown" category)

umetrics_sed_support <- umetrics_sed_support %>% 
    mutate(source_cat = case_when(srceprim %in% c("A", "B") ~ "Fellowship Grant",
                                  srceprim == "C" ~ "Teaching \nAssistantship", 
                                  srceprim == "D" ~ "Research \nAssistantship",
                                  srceprim %in% c("H", "I", "J") ~ "Own Funds/Loan", 
                                  srceprim %in% c("E", "F", "G", "K", "L", "M","N") ~ "Other",
                                  srceprim == '' ~ 'Unknown'))

We can now find out how many people are found on a grant or not by the primary source of support category. Note that we will round the individual counts for the disclosure review purposes and add institutional counts as well.

In [ ]:
# Get the institution count and calculate the rounded counts for the disclosure review
umetrics_sed_support_counts <- umetrics_sed_support %>%
                                    group_by(source_cat, in_umetrics) %>%
                                    summarize(indiv_count = n_distinct(drf_id),
                                             inst_count = n_distinct(phdinst),
                                             indiv_count_rounded = case_when(indiv_count > 999 ~ round(indiv_count, digits = -2),  # round counts
                                                                           indiv_count < 999 ~ round(indiv_count, digits = -1))) # according to rules

In [ ]:
umetrics_sed_support_counts

In [ ]:
# Add a bar plot
ggplot(umetrics_sed_support_counts, aes(x=source_cat, y=indiv_count_rounded, fill=in_umetrics)) + geom_bar(position='dodge',stat='identity')

We can observe that the category with the smallest number of individuals on a grant corresponds to those who indicated own funds/loan as their primary category. We can also see that while more students are not found on a grant in each category, in the Research Assistantship category the differences in the counts are the least pronounced.

This might still be a somewhat surprising -- after all, we might expect those who indicated a primary source of support besides "Fellowship Grant" or "Research Assistantship" to not have received much grant funding. We can check the number of semesters funded for everyone who received some funding within each category to investigate this further.

In [ ]:
# Create a dataframe with only those who are found in UMETRICS on a grant
in_umetrics <- umetrics_sed_support %>% filter(in_umetrics == 'on a grant')

In [ ]:
# Count the number of semesters and create a variable called "sem_funded"
in_umetrics <- in_umetrics %>%
                    group_by(source_cat,drf_id,phdinst) %>%
                    summarize(sem_funded = n_distinct(semester))

In [ ]:
# Box plot
ggplot(in_umetrics, aes(x = source_cat, y = sem_funded)) + geom_boxplot() # x-axis with the source of support, y-axis with the number of semesters

It looks like people who indicated that their own funds or loans were their primary source of support had fewer semesters of support. 

This may seem obvious, but remember: this is an exercise to check data quality and think about what these variables are measuring. The vast majority of people who received grant funding reported either research assistantship or grant fellowships as their primary source of support. However, we do see some people who reported either their own funding or other sources of funding as their primary source of support, even though they had multiple semesters of funding according to UMETRICS.

Why might this be the case? There might be many reasons for this, and it is up to you to think critically about what might make sense. It could be that this is due to **measurement error**, in which respondents to the survey answer incorrectly (for example, because they do not know the source of support while working under their advisor). It could be because of the amount of money involved -- we simply looked at semesters of funding instead of dollar amounts. What do you think can be done to address these?

### Saving Results

In [ ]:
# We will save the dataframe with those individuals who received funding as a CSV file for the visualization notebook
in_umetrics %>% write_csv('Tables\\Source_of_support_UMETRICS.csv')